In [1]:
import sys
import argparse

sys.path.append('../src-py/')

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
from mt_bart import BartForMultiTaskArgGeneration

from utils import *

In [3]:
import transformers
import datasets

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

Running on transformers v4.9.1 and datasets v1.10.2


In [4]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

In [5]:
train_ds = Dataset.from_pandas(pd.read_pickle('../data/train.pkl'))
valid_ds = Dataset.from_pandas(pd.read_pickle('../data/valid.pkl'))

In [6]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForMultiTaskArgGeneration.from_pretrained('facebook/bart-base', without_classification_head=True)

False


In [8]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "../data/output/wp_model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

In [9]:
tmp_ds = train_ds.train_test_split(0.1)
train_ds = tmp_ds['test']

tmp_ds = valid_ds.train_test_split(0.1)
valid_ds = tmp_ds['test']

In [11]:
train_ds = train_ds.map(lambda x :process_instance(x, tokenizer))
valid_ds = valid_ds.map(lambda x :process_instance(x, tokenizer))

  0%|          | 0/9043 [00:00<?, ?ex/s]

  0%|          | 0/2994 [00:00<?, ?ex/s]

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForMultiTaskArgGeneration.forward` and have been ignored: premises, conclusion, counter, weak_premises.
***** Running training *****
  Num examples = 9043
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 849
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TypeError: zip argument #1 must support iteration